# 七節　温水暖房

## 付録B　ガス温水暖房機及びガス給湯温水暖房機

In [1]:
from enum import Enum

In [2]:
# 式（1）
def get_E_G_hs(_Q_out_H_hs, _Q_body, _e_ex):
    if _Q_out_H_hs == 0:
        return 0
    elif _Q_out_H_hs > 0:
        return (_Q_out_H_hs + _Q_body) / _e_ex


#### Example

In [3]:
get_E_G_hs(10,25,0.8)

43.75

In [4]:
# 式（2）
def get_Q_body(_equipment, _supply_theta = None):
    if _equipment == 'gasConv':
        return 240.96 * 3600 * 10**(-6)
    elif _equipment == 'gasCondensing' and _supply_theta == 60:
        return 225.26 * 3600 * 10**(-6)
    elif _equipment == 'gasCondensing' and _supply_theta == 40:
        return 123.74 * 3600 * 10**(-6)

#### Example

In [5]:
gasConv = get_Q_body(_equipment = 'gasConv')
gasCondensing_60 = get_Q_body(_equipment = 'gasCondensing', _supply_theta = 60)
gasCondensing_40 = get_Q_body(_equipment = 'gasCondensing', _supply_theta = 40)
gasConv, gasCondensing_60, gasCondensing_40

(0.867456, 0.810936, 0.44546399999999997)

In [6]:
# 式（3）
def get_e_ex(_equipment, _e_rtd, _q_rtd_hs, _Q_body, _supply_theta = None):
    
    # f_rtd
    if _equipment == 'gasConv':
        _f_rtd = 0.985
    elif _equipment == 'gasCondensing' and _supply_theta == 60:
        _f_rtd = 1.038
    elif _equipment == 'gasCondensing' and _supply_theta == 40:
        _f_rtd = 1.064
    
    return _e_rtd * _f_rtd * (_q_rtd_hs * 3600 * 10**(-6) + _Q_body) / (_q_rtd_hs * 3600 * 10**(-6))
    

#### Example

In [7]:
get_e_ex(_equipment = 'gasCondensing', _e_rtd = 0.81, _q_rtd_hs = 14000, _Q_body = 0.85, _supply_theta = 60)

0.8549598214285715

In [8]:
class HeatRadiator_MainRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2

class HeatRadiator_OtherRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2
    

In [9]:
# 付録H　温水暖房用熱源機の最大能力

# 暖冷房区画の床面積には、温水暖房により暖房される暖冷房区画のみを積算する。

# 主居室、その他居室の放熱器の設置有無　＊　運転方法

def get_q_max_hs(_region, _A_HCZ_i, _HeatRadiator_MainRoom_Exist, _HeatRadiator_OtherRoom_Exist, _HeatRadiator_MainRoom_Operation, _HeatRadiator_OtherRoom_Operation):
    
    _f_CT = 1.05
    
    # _q_rq_H
    if _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'BothRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.NotExist:
        _HeatRadiator_Exist = 'MainRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.NotExist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'OtherRoom_Exist'
    
    coef = {'Region1' : {'BothRoom_Exist' : 90.02, 'MainRoom_Exist' : 139.26, 'OtherRoom_Exist' : 62.28},
            'Region2' : {'BothRoom_Exist' : 77.81, 'MainRoom_Exist' : 120.65, 'OtherRoom_Exist' : 53.26}        
    }
    _q_rq_H = coef[_region][_HeatRadiator_Exist]
    
    # _f_CI
      # 主たる居室の運転方法、その他の居室の運転方法
    _f_CI = {'Continuous'   : {'Continuous' : 1.0 , 'Intermittent' : 1.0 , 'NotExist' : 1.0 },
             'Intermittent' : {'Continuous' : 2.25, 'Intermittent' : 2.25, 'NotExist' : 3.03},
             'NotExist'     : {'Continuous' : 1.0 , 'Intermittent' : 1.62}
    }[_HeatRadiator_MainRoom_Operation][_HeatRadiator_OtherRoom_Operation]
    
    print(_q_rq_H , sum(_A_HCZ_i) , _f_CT , _f_CI)
    
    return _q_rq_H * sum(_A_HCZ_i) * _f_CT * _f_CI
    
    
    
    
    
    

#### Example

In [10]:
get_q_max_hs(_region = 'Region2', _A_HCZ_i = (29.81,0), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.NotExist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.Exist, _HeatRadiator_MainRoom_Operation = 'Intermittent', _HeatRadiator_OtherRoom_Operation = 'Intermittent')

53.26 29.81 1.05 2.25


3750.8954175000003

In [11]:
# 式（4）
def get_E_E_hs(_E_E_hs_pmp, _E_E_hs_fan):
    return _E_E_hs_pmp + _E_E_hs_fan

In [12]:
# 式（5）
# _r_WS_hsは4章本編の話と思われる。
def get_E_E_hs_pmp(_r_WS_hs):
    _P_hs_pmp = 73 # Unit : W
    return _P_hs_pmp * _r_WS_hs * 10**(-3)

In [13]:
# 式（6）
def get_E_E_hs_fan(_E_G_hs):
    r = 0.003
    return _E_G_hs * r * 10**(-3) / 3600
    

In [14]:
# 式（7）
def get_Q_max_H_hs(_region, _A_HCZ_i, _HeatRadiator_MainRoom_Exist, _HeatRadiator_OtherRoom_Exist, _HeatRadiator_MainRoom_Operation, _HeatRadiator_OtherRoom_Operation):
    
    q_rtd_hs = get_q_max_hs(_region, _A_HCZ_i, _HeatRadiator_MainRoom_Exist, _HeatRadiator_OtherRoom_Exist, _HeatRadiator_MainRoom_Operation, _HeatRadiator_OtherRoom_Operation)
    
    return q_rtd_hs * 3600 / 10**(6)

In [16]:
get_Q_max_H_hs(_region='Region1', _A_HCZ_i=(29,81, 10), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.Exist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.NotExist, _HeatRadiator_MainRoom_Operation = 'Intermittent', _HeatRadiator_OtherRoom_Operation = 'NotExist'):

SyntaxError: invalid syntax (<ipython-input-16-2362886f7f5b>, line 1)